Terminal -> New Terminal

python3 -m venv venv

source venv/bin/activate

Select Kernel -> venv

pip install evtx

pip freeze > requirements.txt

pip install -r requirements.txt (quando for instalar)

In [16]:
import os

In [17]:
#cwd (current working directory)
caminho_para_diretorio_atual = os.getcwd()
caminho_para_diretorio_atual

'c:\\Users\\natha\\Desktop\\git\\scientific-research'

In [18]:
caminho_para_arquivos_evtx = "C:/Users/natha/Desktop/ic/logs"
#chdir (change directory)
os.chdir(caminho_para_arquivos_evtx)
os.getcwd()

'C:\\Users\\natha\\Desktop\\ic\\logs'

In [19]:
import glob
for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    print(nome_do_arquivo_evtx)

Application.evtx
System.evtx


In [20]:
import sys
print(sys.executable)

C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [21]:
import sys
sys.path.append('C:\\Users\\natha\\Desktop\\git\\scientific-research\\venv\\Lib\\site-packages')


In [22]:
import glob
from evtx import PyEvtxParser

for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    caminho_do_arquivo_evtx = caminho_para_arquivos_evtx + "\\" + nome_do_arquivo_evtx
    conjunto_de_registros_do_evtx = PyEvtxParser(caminho_do_arquivo_evtx)
    
    for registro in conjunto_de_registros_do_evtx.records():
        print(registro)
        break

{'event_record_id': 127342, 'timestamp': '2023-09-02 18:04:26.708071 UTC', 'data': '<?xml version="1.0" encoding="utf-8"?>\n<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="Microsoft-Windows-Security-SPP" Guid="{E23B33B0-C8C9-472C-A5F9-F2BDFEA0F156}" EventSourceName="Software Protection Platform Service">\n    </Provider>\n    <EventID Qualifiers="49152">16394</EventID>\n    <Version>0</Version>\n    <Level>4</Level>\n    <Task>0</Task>\n    <Opcode>0</Opcode>\n    <Keywords>0x80000000000000</Keywords>\n    <TimeCreated SystemTime="2023-09-02T18:04:26.708071Z">\n    </TimeCreated>\n    <EventRecordID>127342</EventRecordID>\n    <Correlation>\n    </Correlation>\n    <Execution ProcessID="0" ThreadID="0">\n    </Execution>\n    <Channel>Application</Channel>\n    <Computer>Nathan</Computer>\n    <Security>\n    </Security>\n  </System>\n  <EventData>\n    <Data></Data>\n    <Binary></Binary>\n  </EventData>\n</Event>'}
{'event_record

In [23]:
registro['data']

'<?xml version="1.0" encoding="utf-8"?>\n<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="e1i65x64">\n    </Provider>\n    <EventID Qualifiers="24580">32</EventID>\n    <Version>0</Version>\n    <Level>4</Level>\n    <Task>0</Task>\n    <Opcode>0</Opcode>\n    <Keywords>0x80000000000000</Keywords>\n    <TimeCreated SystemTime="2023-06-28T03:25:29.947705Z">\n    </TimeCreated>\n    <EventRecordID>251109</EventRecordID>\n    <Correlation>\n    </Correlation>\n    <Execution ProcessID="4" ThreadID="264">\n    </Execution>\n    <Channel>System</Channel>\n    <Computer>Nathan</Computer>\n    <Security>\n    </Security>\n  </System>\n  <EventData>\n    <Data>,Intel(R) 82579V Gigabit Network Connection</Data>\n    <Binary>0000040002003000000000002000046000000000000000000000000000000000000000000000000020000460</Binary>\n  </EventData>\n</Event>'

O código busca uma string que começa com <Level>, seguida de qualquer conteúdo (capturado por (.*?)), e termina com </Level> dentro da string registro['data'].

In [24]:
import re
re.search(r'<Level>(.*?)</Level>', registro['data'])

<re.Match object; span=(241, 257), match='<Level>4</Level>'>

Esta parte do código acessa o primeiro grupo de captura ((.*?)) do objeto de correspondência retornado por re.search(). Em expressões regulares, os grupos de captura são definidos colocando partes do padrão entre parênteses. Como (.*?) é o primeiro (e único) grupo de captura no padrão, .group(1) recupera o conteúdo correspondido por este grupo.

In [25]:
re.search(r'<Level>(.*?)</Level>', registro['data']).group(1)

'4'

In [26]:
event_level = re.search(r'<Level>(.*?)</Level>', registro['data']).group(1)
event_level

'4'

# Event_level 1 e 2 são falhas

In [27]:
# if((("Application" in nome_do_arquivo_evtx) & (event_level == "2")) | (("System" in nome_do_arquivo_evtx) & ((event_level == "2") | (event_level == "1")))):

Como refatorar o código

In [28]:
# conditions = [
#     ("Application" in nome_do_arquivo_evtx and event_level == "2"),
#     ("System" in nome_do_arquivo_evtx and event_level in ("1", "2"))
# ]

# if any(conditions):
#     # Your code here


### Eventos que não saõ de falhas, mas possuem Event_level 1 e 2

In [29]:
import pandas as pd

os.chdir(caminho_para_diretorio_atual)
df_non_failure_events = pd.read_csv("1_non_failure_events.csv", encoding='utf-8-sig', engine='python')
df_non_failure_events

,SourceName,EventID
0,trend micro officescan,900
1,eventlog,6008
2,tcpip,4199
3,server,2505
4,netbt,4321
5,netbt,4319
6,microsoft-windows-kernel-power,41


In [30]:
from collections import defaultdict


dict_non_failure_events = defaultdict(list)
for k, v in zip(df_non_failure_events.SourceName, df_non_failure_events.EventID):
    dict_non_failure_events[k].append(v)
dict_non_failure_events

defaultdict(list,
            {'trend micro officescan': [900],
             'eventlog': [6008],
             'tcpip': [4199],
             'server': [2505],
             'netbt': [4321, 4319],
             'microsoft-windows-kernel-power': [41]})

# Código Completo

In [31]:
class Event(object):
    def __init__(
        self,
        Filename,
        ID,
        SourceName,
        Provider_Guid,
        EventID,
        EventID_Qualifiers,
        Version,
        Level,
        Task,
        Opcode,
        Keywords,
        TimeCreated,
        EventRecordID,
        Correlation_ActivityID,
        Correlation_RelatedActivityID,
        ProcessID,
        ThreadID,
        Channel,
        Computer,
        UserID,
        Data_Event_Name,
        Data_Fields,
        Data,
        UserData_Flag
    ):
        self.Filename = Filename
        self.ID = ID
        self.SourceName = SourceName
        self.Provider_Guid = Provider_Guid
        self.EventID = EventID
        self.EventID_Qualifiers = EventID_Qualifiers
        self.Version = Version
        self.Level = Level
        self.Task = Task
        self.Opcode = Opcode
        self.Keywords = Keywords
        self.TimeCreated = TimeCreated
        self.EventRecordID = EventRecordID
        self.Correlation_ActivityID = Correlation_ActivityID
        self.Correlation_RelatedActivityID = Correlation_RelatedActivityID
        self.ProcessID = ProcessID
        self.ThreadID = ThreadID
        self.Channel = Channel
        self.Computer = Computer
        self.UserID = UserID
        self.Data_Event_Name = Data_Event_Name
        self.Data_Fields = Data_Fields
        self.Data = Data
        self.UserData_Flag = UserData_Flag

In [32]:
def parseXMLtoString(root, string_search):
    result = root.xpath(string_search)
    if  ("Data" not in string_search) and (result):
        result = str(result[0])

    return result

In [33]:
from lxml import etree # parsing de xml
import xmltodict # parsing de xml

os.chdir(caminho_para_arquivos_evtx)
listaDe_eventos_dos_arquivos_evtx = []
i = 0
UserData_Flag = 0

for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    # conjunto_de_registros_do_evtx = PyEvtxParser(caminho_para_arquivos_evtx+nome_do_arquivo_evtx) # comentar essa linha e descomentar as debaixo
    caminho_do_arquivo_evtx = caminho_para_arquivos_evtx + "\\" + nome_do_arquivo_evtx
    conjunto_de_registros_do_evtx = PyEvtxParser(caminho_do_arquivo_evtx)
    for registro in conjunto_de_registros_do_evtx.records():
        event_level = re.search(r'<Level>(.*?)</Level>', registro['data']).group(1) # pegar o level de cada registro no campo data
        event_id = re.search(r'<EventID(?:[^>]*)>(\d+)</EventID>', registro['data']).group(1) # o mesmo para o ID, group(1) pega o numero dentro dos <><>
        #Event_level 1 e 2 são falhas



        conditions = [
            ("Application" in nome_do_arquivo_evtx and event_level == "2"),
            ("System" in nome_do_arquivo_evtx and event_level in ("1", "2"))
        ]
        
        if any(conditions):
            
            i = i + 1
            Filename = nome_do_arquivo_evtx.split("_")[0] #pega o que está a esquerda de "_"

            #Em produção
            # Filename = nome_do_arquivo_evtx


            ID = i

            #Removendo "<?xml version="1.0" encoding="utf-8"?>" do evento
            line_filtered_1 = re.sub(r'<\?xml([\s\S]*?)>\n','',registro['data']) # argumento 1 é o padrão de busca, argumento 2 é o que substituir o padrão de busca, argumento 3 é a string
            #Removendo "xmlns="http://schemas.microsoft.com/win/2004/08/events/event"" do evento
            line_filtered_2 = re.sub(r' xmlns=\"([\s\S]*?)\"','',line_filtered_1)

            try:
                evento_estrutura_xml = etree.XML(line_filtered_2) # transforma a string em um objeto XML

                evento_estrutura_dict = xmltodict.parse(line_filtered_2)


            except Exception as e:
                
                with open(r"C:\Users\natha\Desktop\git\scientific-research\exception.txt", 'a', encoding='utf-8') as f: # a é append, adiciona ao final do arquivo
                    f.write(nome_do_arquivo_evtx+"_"+str(ID)+"_"+str(e))
                    f.write("\n")
                    continue
            else:
                evento_estrutura_dict__tag_System = evento_estrutura_dict.get("Event").get("System") # pega o dicionário do System

                SourceName = evento_estrutura_dict__tag_System.get("Provider").get("@Name") # pega o valor do campo Provider Name (Service Control Manager)
                if(SourceName is None):
                    SourceName = evento_estrutura_dict__tag_System.get("Provider").get("@EventSourceName") # Se não tem o campo Provider Name, pega o valor do campo EventSourceName


                Provider_Guid = evento_estrutura_dict__tag_System.get("Provider").get("@Guid") # pega o valor do campo Guid
                
                #
                EventID = evento_estrutura_dict__tag_System.get("EventID") # pega o valor do campo EventID
                EventID_Qualifiers = None
                if (type(EventID) != str): # se o EventID não for uma string
                    EventID = evento_estrutura_dict__tag_System.get("EventID").get("#text") # pega o valor do campo EventID
                    EventID_Qualifiers = evento_estrutura_dict__tag_System.get("EventID").get("@Qualifiers") # pega o valor do campo Qualifiers

                # Removendo elementos que não são de falha.
                if(SourceName.lower() in dict_non_failure_events.keys()):
                    listaDe_EventIDs = dict_non_failure_events[SourceName.lower()] # PEDIR PRA EXPLICAR
                    if(int(EventID) in listaDe_EventIDs):
                        continue

                Version = evento_estrutura_dict__tag_System.get("Version")
                Level = evento_estrutura_dict__tag_System.get("Level")
                Task = evento_estrutura_dict__tag_System.get("Task")
                Opcode = evento_estrutura_dict__tag_System.get("Opcode")
                Keywords = evento_estrutura_dict__tag_System.get("Keywords")
                TimeCreated = evento_estrutura_dict__tag_System.get("TimeCreated").get("@SystemTime")
                EventRecordID = evento_estrutura_dict__tag_System.get("EventRecordID")
                
                correlation = evento_estrutura_dict__tag_System.get("Correlation")
                Correlation_ActivityID = None
                Correlation_RelatedActivityID = None
                if(correlation is not None):
                    Correlation_ActivityID = evento_estrutura_dict__tag_System.get("Correlation").get("@ActivityID")
                    Correlation_RelatedActivityID = evento_estrutura_dict__tag_System.get("Correlation").get("@RelatedActivityID")

                execution = evento_estrutura_dict__tag_System.get("Execution")
                ProcessID = None
                ThreadID = None
                if(execution is not None):
                    ProcessID = evento_estrutura_dict__tag_System.get("Execution").get("@ProcessID")
                    ThreadID = evento_estrutura_dict__tag_System.get("Execution").get("@ThreadID")

                Channel = evento_estrutura_dict__tag_System.get("Channel")
                Computer = evento_estrutura_dict__tag_System.get("Computer")

                security = evento_estrutura_dict__tag_System.get("Security")
                UserID = None
                if(security is not None):
                    UserID = evento_estrutura_dict__tag_System.get("Security").get("@UserID")

                dataTag = evento_estrutura_xml.getchildren()[1].tag

                if (dataTag == 'ProcessingErrorData'):
                        Data_Fields = list(evento_estrutura_dict.get("Event").get("ProcessingErrorData").keys()) # pega os campos do ProcessingErrorData
                        Data = list(evento_estrutura_dict.get("Event").get("ProcessingErrorData").values()) # pega os valores do ProcessingErrorData
                        UserData_Flag = 0 
                        Data_Event_Name = None
            if (dataTag == 'EventData'): 
                #Data_Fields = list(evento_estrutura_dict.get("Event").get("UserData").get(list(evento_estrutura_dict.get("Event").get(dataTag).keys())[0]).keys()) # isso
                event_data = evento_estrutura_dict.get("Event")
                user_data = event_data.get("UserData")
                dataTag_keys = list(event_data.get(dataTag).keys())
                primeira_dataTag_key = dataTag_keys[0]

                if(user_data is None):
                    Data_Fields = None
                    Data = None
                else:
                    user_data_with_first_key = user_data.get(primeira_dataTag_key)
                    Data_Fields = list(user_data_with_first_key.keys())
                    Data = list(evento_estrutura_dict.get("Event").get("UserData").get(list(evento_estrutura_dict.get("Event").get(dataTag).keys())[0]).values())
                    UserData_Flag = 1
                    Data_Event_Name = None
            else:
                UserData_Flag = 0
                Data_Event_Name = None
                evento_estrutura_dictEventData = evento_estrutura_dict.get("Event").get("EventData")
                if(evento_estrutura_dictEventData is None):
                    Data_Fields = None
                    Data = None
                else:
                    Data_Fields = (parseXMLtoString(evento_estrutura_xml, ".//Data/@Name"))
                    Data = (parseXMLtoString(evento_estrutura_xml, ".//Data/text()"))

                    if(len(evento_estrutura_dictEventData.keys()) > 1):
                        if(list(evento_estrutura_dictEventData.keys())[1] == "Binary"):
                            Data_Fields = ["Data", "Binary"]
                            Data.append(evento_estrutura_dictEventData.get("Binary"))
                        else:
                            Data_Event_Name = evento_estrutura_dictEventData.get("@Name")            

                evento_unico = Event(
                                Filename,
                                ID,
                                SourceName,
                                Provider_Guid,
                                EventID,
                                EventID_Qualifiers,
                                Version,
                                Level,
                                Task,
                                Opcode,
                                Keywords,
                                TimeCreated,
                                EventRecordID,
                                Correlation_ActivityID,
                                Correlation_RelatedActivityID,
                                ProcessID,
                                ThreadID,
                                Channel,
                                Computer,
                                UserID,
                                Data_Event_Name,
                                Data_Fields,
                                Data,
                                UserData_Flag
                            )
                # print(evento_unico)
                listaDe_eventos_dos_arquivos_evtx.append(evento_unico.__dict__) #

In [34]:
len(listaDe_eventos_dos_arquivos_evtx)

561

In [35]:
df_eventos = pd.DataFrame.from_records(listaDe_eventos_dos_arquivos_evtx)
df_eventos.head()

,Filename,ID,SourceName,Provider_Guid,EventID,EventID_Qualifiers,Version,Level,Task,Opcode,...,Correlation_RelatedActivityID,ProcessID,ThreadID,Channel,Computer,UserID,Data_Event_Name,Data_Fields,Data,UserData_Flag
0,Application.evtx,28,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12320,6132,Application,Nathan,S-1-5-18,None,None,None,0
1,Application.evtx,29,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12320,6132,Application,Nathan,S-1-5-18,None,None,None,0
2,Application.evtx,30,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3011,None,0,2,0,0,...,None,12320,6132,Application,Nathan,S-1-5-18,None,None,None,0
3,Application.evtx,173,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12248,10252,Application,Nathan,S-1-5-18,None,None,None,0
4,Application.evtx,174,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12248,10252,Application,Nathan,S-1-5-18,None,None,None,0


In [36]:
df_eventos.tail()

,Filename,ID,SourceName,Provider_Guid,EventID,EventID_Qualifiers,Version,Level,Task,Opcode,...,Correlation_RelatedActivityID,ProcessID,ThreadID,Channel,Computer,UserID,Data_Event_Name,Data_Fields,Data,UserData_Flag
556,Application.evtx,1485,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,4884,5404,Application,Nathan,S-1-5-18,None,None,None,0
557,Application.evtx,1486,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3011,None,0,2,0,0,...,None,4884,5404,Application,Nathan,S-1-5-18,None,None,None,0
558,Application.evtx,1492,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,10372,10380,Application,Nathan,S-1-5-18,None,None,None,0
559,Application.evtx,1493,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,10372,10380,Application,Nathan,S-1-5-18,None,None,None,0
560,Application.evtx,1494,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3011,None,0,2,0,0,...,None,10372,10380,Application,Nathan,S-1-5-18,None,None,None,0
